# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(filepath,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
full_data_rows_list = [] 
    for f in file_path_list: 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:  
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [ ]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [ ]:
query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(  sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

                    

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_app_history ( sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
query = "select artist, song, length from music_app_history WHERE sessionid = 338 AND iteminsession = 4 "
rows = session.execute(query)

for row in rows:
    print (row.artist, row.song, row.length )

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [ ]:
query1 = "CREATE TABLE IF NOT EXISTS music_app_history1 "
query1 = query1 + "( userId int, sessionId int,  itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)  

                    

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO music_app_history1 ( userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query1, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
query1 = "select artist, song, firstname, lastname from music_app_history1 WHERE userid = 10 AND sessionid = 182 "
rows = session.execute(query1)

for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname )


In [ ]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row = pandas_factory
session.default_fetch_size = None

query1 = "select artist, song, firstname, lastname from music_app_history1 WHERE userid = 10 AND sessionid = 182 "
rows = session.execute(query1)

for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname )

df = rows._current_rows
print(df)

In [ ]:
query2 = "CREATE TABLE IF NOT EXISTS music_app_history2 "
query2 = query2 + "( song text, firstName text, lastName text, PRIMARY KEY (song))"
try:
    session.execute(query2)
except Exception as e:
    print(e)  

                    

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query2 = "INSERT INTO music_app_history2 ( song, firstName, lastName)"
        query2 = query2 + " VALUES (%s, %s, %s)"
        session.execute(query2, ( line[9], line[1], line[4]))

In [ ]:
query2 = "select firstname, lastname from music_app_history2 WHERE song = 'All Hands Against His Own' "
rows = session.execute(query2)

for row in rows:
    print (row.firstname, row.lastname )

### Drop the tables before closing out the sessions

In [ ]:
query = "drop table music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query1 = "drop table music_app_history1"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
query1 = "drop table music_app_history2"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()